In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigs

In [ ]:
Pi = 3.14159265358979323846

def knn_graph(Data, method, k):
    # 获取样本点的数量
    n = Data.shape[0]
    # 初始化k近邻图的邻接矩阵
    knn_adjacency_matrix = np.zeros((n, n))  
    # 计算欧式距离矩阵
    distances = np.sqrt(np.sum((Data[:, None] - Data) ** 2, axis=2))
    if method == 'epsilon':
        return knn_adjacency_matrix, distances
    # 获取每个样本点的最近邻索引
    indices = np.argsort(distances, axis=1)[:, 1:k+1]
    # 构建k近邻图的权重矩阵
    # 遍历每个样本点的最近邻索引
    for i in range(n):
        knn_adjacency_matrix[i, indices[i]] = 1
        knn_adjacency_matrix[indices[i], i] = 1
    return knn_adjacency_matrix, distances

# 以每个点到其他所有点的平均值作为每个数据点的平均邻域半径
def compute_avg_radius(n, distances): 
    radius = np.zeros(n) # 存储每个数据点的平均邻域半径
    for i in range(n): # 计算每个数据点的平均邻域半径
        avg_radius = np.mean(distances[i]) # 每个数据点到其他所有点的平均值
        radius[i] = avg_radius # 存储每个数据点的平均邻域半径
    return radius

# 以将距离从小到大排序后的第k个距离作为每个数据点的半径
def compute_k_maximun_radius(n, distances, k):
    radius = np.zeros(n)
    sorted_distances = np.sort(distances, axis=1)  # 对距离矩阵的每一行进行排序
    for i in range(n):
        # 选取第k个距离作为半径
        radius[i] = sorted_distances[i, k]
    return radius

# 计算k最近邻接矩阵或epsilon邻接矩阵
def compute_neighborhood_matrix(Data, method, k):
    n = len(Data)
    knn_adjacency_matrix, distances = knn_graph(Data, method, k)
    if method == 'knn':
        return knn_adjacency_matrix, distances
    elif method == 'epsilon':
        adjacency_matrix = np.zeros((n, n))
        radius = compute_k_maximun_radius(n, distances, k)  # 计算每个数据点的邻域半径
        for i in range(n):  # 对于数据集中的每个样本点 i
            neighbors = np.where(distances[i] <= radius[i])[0]  # 获取epsilon邻域内的样本索引
            adjacency_matrix[i, neighbors] = 1
            adjacency_matrix[neighbors, i] = 1
        return adjacency_matrix, distances

# 构建基于热核方法的权重矩阵
def construct_weight_matrix(Data, method, k, t):
    n = len(Data)
    Weight_matrix = np.zeros((n, n))
    adjacency_matrix, distances = compute_neighborhood_matrix(Data, method, k)
    # 计算相似度矩阵
    similarity_matrix = np.exp(-distances ** 2 / t)
    # 将相似度矩阵按照邻接矩阵进行筛选，得到需要设置权重的位置
    i_indices, j_indices = np.where(adjacency_matrix == 1)
    # 设置权重
    Weight_matrix[i_indices, j_indices] = similarity_matrix[i_indices, j_indices]
    Weight_matrix[j_indices, i_indices] = similarity_matrix[i_indices, j_indices]  # 对称矩阵
    # 计算全局相似度
    '''
    修正权重矩阵的原理是利用全局相似度来修正局部相似度得到的权重矩阵，以使得整个权重矩阵更加平滑和连续。
    '''
    Weight_matrix += np.exp(-distances ** 2 / t)
    return Weight_matrix   


def LPP_old(Data, d, method, k, t):
    # Step 1: 计算权重矩阵
    Weight_matrix = construct_weight_matrix(Data, method, k, t)
    print("权重矩阵形状：", Weight_matrix.shape)
    # Step 2: 计算度矩阵和拉普拉斯矩阵
    Degree_matrix = np.diag(np.sum(Weight_matrix, axis=1))
    print("度矩阵形状：", Degree_matrix.shape)
    Laplacian_matrix = Degree_matrix - Weight_matrix
    print("拉普拉斯矩阵形状：", Laplacian_matrix.shape)
    # Step 3: 进行特征映射
    eigenvalues, eigenvectors = eigs(Laplacian_matrix, k=d+1, which='SR')
    sorted_indices = np.argsort(eigenvalues.real)
    selected_indices = sorted_indices[1:d + 1]
    selected_eigenvectors = eigenvectors.real[:, selected_indices]
    return selected_eigenvectors

def LPP(train_data, train_labels, method, d, k, t):
    Data = train_data.T
    n = len(train_labels)
    Weight_matrices = np.zeros((n, n)) # 存储每个类别的权重矩阵
    Degree_matrices = np.zeros((n, n)) # 存储每个类别的度矩阵
    for class_label in np.unique(train_labels):
        class_indices = np.where(train_labels == class_label)[0]  # 获取当前类别的样本索引
        class_train_data = Data[:, class_indices]  # 获取当前类别的样本数据
        Weight_matrix = construct_weight_matrix(class_train_data, method, k, t)  # 计算当前类别的权重矩阵
        Degree_matrix = np.diag(np.sum(Weight_matrix, axis=1))  # 计算当前类别的度矩阵
        # 将Degree_matrix和Weight_matrix计算矩阵的迹并添加到对角线
        class_idx = class_label - 1
        Weight_matrices[class_idx, class_idx] = np.trace(Weight_matrix)
        Degree_matrices[class_idx, class_idx] = np.trace(Degree_matrix)
    Laplacian_matrices = Degree_matrices - Weight_matrices
    print("拉普拉斯矩阵形状：", Laplacian_matrices.shape)
    objective_value = np.dot(np.dot(train_data, Laplacian_matrices), train_data.T)
    print("目标函数形状：", objective_value.shape)
    eigenvalues, eigenvectors = eigs(objective_value, k=d+1, which='SR')
    sorted_indices = np.argsort(eigenvalues.real)
    selected_indices = sorted_indices[1:d + 1]  
    selected_eigenvectors = eigenvectors.real[:, selected_indices]
    print("特征向量形状：", selected_eigenvectors.shape) 
    return selected_eigenvectors

In [ ]:
def read_images(dataset_dir, target_size=(32, 32)):
    data = []  # 存储图像数据的列表
    labels = []  # 存储标签的列表
    faceshape = [] # 存储图像形状
    for class_dir in os.listdir(dataset_dir):  # 遍历数据集文件夹中的文件夹（每个文件夹代表一个类别）
        class_path = os.path.join(dataset_dir, class_dir)  # 类别文件夹路径
        for file_name in os.listdir(class_path):  # 遍历每个类别文件夹中的图像文件
            file_path = os.path.join(class_path, file_name)  # 图像文件路径
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # 读取灰度图像
            # 缩放图像至目标尺寸
            img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
            # 读取第一张灰度图像的大小作为图片形状
            faceshape = img.shape
            data.append(img.flatten())  # 将图像展平并添加到数据列表中
            labels.append(int(class_dir))  # 将类别标签添加到标签列表中
    return np.array(data), np.array(labels).reshape(-1, 1), faceshape  # 返回图像数据和标签

# 训练集和测试集划分
def train_test_split(data, labels, train_test_split_ratio):
    num_samples = data.shape[0]  # 总样本数
    train_samples = int(num_samples * train_test_split_ratio)  # 训练集样本数
    
    # 洗牌算法打乱数据集
    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    data = data[indices]
    labels = labels[indices]
    
    # 划分训练集和测试集
    train_data = data[:train_samples]
    train_labels = labels[:train_samples]
    test_data = data[train_samples:]
    test_labels = labels[train_samples:]
    
    return train_data, train_labels, test_data, test_labels

In [ ]:
def ShowEigenface(eigenfaces, faceshape):
    # 显示前16个特征脸
    fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(8, 10))
    for i in range(16):
        axes[i%4][i//4].imshow(eigenfaces[:, i].reshape(faceshape), cmap="gray")
    #print("显示特征脸")
    plt.show()

In [ ]:
# 读取人脸图像
faces, classes, faceshape = read_images("attfaces")  # 读取图像数据和标签

train_data, train_labels, test_data, test_labels = train_test_split(faces, classes, 0.5)  # 划分训练集和测试集
# 打印一些细节
print("人脸图像形状:", faceshape)
print("类别数量:", len(classes))
print("图像数量:", len(faces))
print("测试集标签形状:", test_labels.shape)

In [ ]:
# 应用LPP并选择前K个主成分作为特征脸

d = 70
k = 135
t = 87250
method = 'knn'
"""
d = 70
k = 450
t = 50000
method = 'epsilon'
"""
train_data = train_data.T
print("训练集形状：", train_data.shape)
#from lpproj import LocalityPreservingProjection
#lpp = LocalityPreservingProjection(k=k, d=d)
#lpp.fit(train_data)
#lpp_eigenfaces = lpp.transform(train_data)



#A = LPP(train_data, d, method, k, t)
#print("A形状：", A.shape)
#lpp_eigenfaces = (np.dot(A.T, train_data.T)).T
lpp_eigenfaces = LPP_old(train_data, d, method, k, t)
ShowEigenface(lpp_eigenfaces, faceshape)
print("特征脸形状:", lpp_eigenfaces.shape)

overall_mean = np.mean(train_data , axis=1).reshape(-1, 1)
print("平均人脸形状:", overall_mean.shape)

lpp_weight_matrix = lpp_eigenfaces.T @ (train_data-overall_mean) 
print("权重矩阵形状:", lpp_weight_matrix.shape)

In [ ]:
def test_image(i, faceshape, overall_mean, train_labels, test_labels, train_data, test_data, lpp_eigenfaces, lpp_weight_matrix):
    # 计算测试图像的权重向量
    #print("测试图像形状:", query.shape)
    #print("平均人脸形状:", overall_mean.shape)
    #print("特征脸形状:", dlpp_eigenfaces.shape)
    query = test_data[i]
    query_weight = (lpp_eigenfaces.T @ (query - overall_mean.flatten()).reshape(-1, 1))
    # 计算测试图像与数据集中每个人脸的欧氏距离
    euclidean_distances = np.linalg.norm(lpp_weight_matrix - query_weight, axis=0)
    # 找到最佳匹配的人脸
    best_match_index = np.argmin(euclidean_distances)
    #判断是否匹配正确
    flag = False
    if train_labels[best_match_index] == test_labels[i]:
        flag = True
    else:
        flag = False
    """
    # 可视化
    fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 6))
    axes[0].imshow(query.reshape(faceshape), cmap="gray")
    axes[0].set_title("Query Image")
    axes[1].set_xlabel("Euclidean Distance: {:.0f}".format(euclidean_distances[best_match_index]))
    axes[1].imshow(train_data[best_match_index].reshape(faceshape), cmap="gray")
    axes[1].set_title("Best Match")
    plt.show()
    """
    return flag

# 识别率统计
wrong_times = 0
right_times = 0
for i in range(test_data.shape[0]):
    flag = test_image(i, faceshape, overall_mean, train_labels, test_labels, train_data, test_data, lpp_eigenfaces, lpp_weight_matrix)
    if flag:
                right_times += 1
    else:
        wrong_times += 1
rate = right_times / test_data.shape[0]
print(f"Recognition Rate: {rate}")

In [ ]:
# 测试集图片分类
test_weights = np.zeros((d, test_data.shape[0]))  
for i in range(test_data.shape[0]):
    test_weights[:, i] = (lpp_eigenfaces.T @ (test_data[i] - overall_mean.flatten())) 
print("测试权重形状:", test_weights.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns

# 计算成对欧式距离矩阵
pairwise_distances = np.zeros((test_data.shape[0], test_data.shape[0]))
for i in range(test_data.shape[0]):
    for j in range(test_data.shape[0]):
        pairwise_distances[i, j] = np.linalg.norm(test_weights[:, i] - test_weights[:, j])
np.savetxt('pairwise_distances.csv', pairwise_distances, delimiter=',')
# 构建KNN分类器
knn_classifier2 = KNeighborsClassifier(n_neighbors=2, metric='precomputed')
knn_classifier2.fit(pairwise_distances, test_labels.ravel())  # 注意将test_labels从二维矩阵转换为一维数组

# 预测测试集图片的标签
predicted_labels2 = knn_classifier2.predict(pairwise_distances)

# 计算分类准确率
accuracy = accuracy_score(test_labels.ravel(), predicted_labels2)
print("KNN分类器准确率:", accuracy)

# 可视化展示分类结果
plt.figure(figsize=(10, 8))
sns.heatmap(pairwise_distances, cmap="viridis", xticklabels=False, yticklabels=False)
plt.title("Pairwise Euclidean Distances Matrix")
plt.xlabel("Test Images")
plt.ylabel("Test Images")
plt.show()


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 初始化KNN分类器
knn_classifier1 = KNeighborsClassifier(n_neighbors=1)
# 训练KNN分类器
knn_classifier1.fit(lpp_weight_matrix.T, train_labels)

# 使用KNN分类器对测试集进行预测
predicted_labels1 = knn_classifier1.predict(test_weights.T)

# 计算分类准确率
accuracy = accuracy_score(test_labels, predicted_labels1)

# 可视化显示分类结果
plt.figure(figsize=(10, 6))
plt.scatter(range(len(predicted_labels1)), predicted_labels1, c='r', marker='x', label='Predicted Labels')
plt.scatter(range(len(test_labels)), test_labels, c='b', marker='o', label='True Labels')
plt.xlabel('Sample Index')
plt.ylabel('Label')
plt.title('Classification Results'+f"(Accuracy: {accuracy})")
plt.legend()
plt.show()

In [ ]:
#transformation_matrix = LPP(train_data, train_labels, method, d, k, t)
#print("变换矩阵形状：", transformation_matrix.shape)
lpp_eigenfaces = LPP(train_data, train_labels, method, d, k, t)
ShowEigenface(lpp_eigenfaces, faceshape)
print("特征脸形状:", lpp_eigenfaces.shape)

In [ ]:
# 测试集图片分类
test_weights = np.zeros((d, test_data.shape[0]))  
for i in range(test_data.shape[0]):
    test_weights[:, i] = (lpp_eigenfaces.T @ (test_data[i] - overall_mean.flatten())) 
print("测试权重形状:", test_weights.shape)

# 构建KNN分类器将测试集的图片进行分类
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(lpp_weight_matrix.T, train_labels.flatten())
predicted_labels = knn.predict(test_weights.T)
# 计算分类准确率
accuracy = np.mean(predicted_labels == test_labels.flatten())
print(f"Classification Accuracy: {accuracy}")
#可视化展示分类结果
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 6))
axes[0].imshow(test_data[0].reshape(faceshape), cmap="gray")
axes[0].set_title("Query Image")
axes[1].set_xlabel("Predicted Label: {}".format(predicted_labels[0]))
axes[1].imshow(train_data.T[0].reshape(faceshape), cmap="gray")
axes[1].set_title("Best Match")
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns

# 计算成对欧式距离矩阵
pairwise_distances = np.zeros((test_data.shape[0], test_data.shape[0]))
for i in range(test_data.shape[0]):
    for j in range(test_data.shape[0]):
        pairwise_distances[i, j] = np.linalg.norm(test_weights[:, i] - test_weights[:, j])

# 构建KNN分类器
knn_classifier = KNeighborsClassifier(n_neighbors=3, metric='precomputed')
knn_classifier.fit(pairwise_distances, test_labels.ravel())  # 注意将test_labels从二维矩阵转换为一维数组

# 预测测试集图片的标签
predicted_labels = knn_classifier.predict(pairwise_distances)

# 计算分类准确率
accuracy = accuracy_score(test_labels.ravel(), predicted_labels)
print("KNN分类器准确率:", accuracy)

# 可视化展示分类结果
plt.figure(figsize=(10, 8))
sns.heatmap(pairwise_distances, cmap="viridis", xticklabels=False, yticklabels=False)
plt.title("Pairwise Euclidean Distances Matrix")
plt.xlabel("Test Images")
plt.ylabel("Test Images")
plt.show()



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 初始化KNN分类器
knn_classifier = KNeighborsClassifier(n_neighbors=4)
# 训练KNN分类器
knn_classifier.fit(lpp_weight_matrix.T, train_labels)

# 使用KNN分类器对测试集进行预测
predicted_labels = knn_classifier.predict(test_weights.T)

# 可视化显示分类结果
plt.figure(figsize=(10, 6))
plt.scatter(range(len(predicted_labels)), predicted_labels, c='r', marker='x', label='Predicted Labels')
plt.scatter(range(len(test_labels)), test_labels, c='b', marker='o', label='True Labels')
plt.xlabel('Sample Index')
plt.ylabel('Label')
plt.title('Classification Results')
plt.legend()
plt.show()

# 计算分类准确率
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Classification Accuracy: {accuracy}")